In [ ]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np

In [ ]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/larger_clap_music")
processor = AutoProcessor.from_pretrained("laion/larger_clap_music")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# Process audio

In [ ]:
import zipfile
import os

zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

extract_dir = "/content/" # You can change this if you want to extract elsewhere
os.makedirs(extract_dir, exist_ok=True)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Error: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


In [ ]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [ ]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [ ]:
print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [ ]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [ ]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
tag_embeds = model.get_text_features(**tag_inputs)

In [ ]:
tag_inputs_disc_i = processor(text=discrete_captions_induced, return_tensors="pt", padding=True)
tag_embeds_disc_i = model.get_text_features(**tag_inputs_disc_i)

In [ ]:
tag_inputs_dim_p = processor(text=dimensional_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_dim_p = model.get_text_features(**tag_inputs_dim_p)

In [ ]:
tag_inputs_disc_p = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_disc_p = model.get_text_features(**tag_inputs_disc_p)

## Load csv files and extract related columns

In [ ]:
IDim_path = '/content/Exp1/Data/IDim/'
IDim_response_dfs = []

for file in sorted(os.listdir(IDim_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(IDim_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive', 'relaxed', 'awake']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDim_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDim_response_dfs:
    master_human_responses_df = pd.concat(IDim_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3835, 3)

Master human responses (first 5 rows):
   positive  relaxed  awake
0      3.68     3.78   4.42
1      5.88     5.98   3.89
2      6.53     5.59   6.59
3      6.26     5.71   6.88
4      2.80     2.62   5.15



In [ ]:
PDim_path = '/content/Exp1/Data/PDim/'
PDim_response_dfs = []

for file in sorted(os.listdir(PDim_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(PDim_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive', 'relaxed', 'awake']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                PDim_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if PDim_response_dfs:
    master_human_responses_df_dim_p = pd.concat(PDim_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_dim_p.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_dim_p.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3953, 3)

Master human responses (first 5 rows):
   positive  relaxed  awake
0      2.23     9.00   2.37
1      6.56     5.64   4.12
2      5.11     6.23   4.04
3      7.14     7.67   2.95
4      1.85     1.77   2.98



In [ ]:
IDisc_path = '/content/Exp1/Data/IDisc/'
IDisc_response_dfs = []

for file in sorted(os.listdir(IDisc_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ["happiness", "sadness", "anger", "tenderness", "fear"]
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_response_dfs:
    master_human_responses_df_disc_i = pd.concat(IDisc_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_disc_i.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_disc_i.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3894, 5)

Master human responses (first 5 rows):
   happiness  sadness  anger  tenderness  fear
0       1.00     6.97   5.01        9.00  7.89
1       1.00     6.00   5.80        6.78  1.00
2       1.00     2.42   5.99        6.44  1.09
3       3.92     6.13   2.25        5.96  1.22
4       1.99     7.92   6.82        5.83  1.39



In [ ]:
PDisc_path = '/content/Exp1/Data/PDisc/'
PDisc_response_dfs = []

for file in sorted(os.listdir(PDisc_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(PDisc_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ["happiness", "sadness", "anger", "tenderness", "fear"]
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                PDisc_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if PDisc_response_dfs:
    master_human_responses_df_disc_p = pd.concat(PDisc_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_disc_p.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_disc_p.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3835, 5)

Master human responses (first 5 rows):
   happiness  sadness  anger  tenderness  fear
0       1.32     1.30   1.27        1.25  3.18
1       1.00     1.44   1.76        1.00  1.24
2       1.00     5.87   1.23        1.28  1.56
3       1.92     3.36   4.82        1.20  2.32
4       1.23     2.31   7.10        1.61  2.86



# Prepare features X and targets y

In [ ]:
from sklearn.model_selection import train_test_split


num_participants = len(IDim_response_dfs)
if master_human_responses_df.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X = np.array(X_list)

# Extract y from the concatenated DataFrame
y = master_human_responses_df[['positive', 'relaxed', 'awake']].values

print(f"Shape of X (features) after implicit alignment: {X.shape}")
print(f"Shape of y (labels) after implicit alignment: {y.shape}\n")

# Sanity check: X and y must have the same number of rows
if X.shape[0] != y.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_train.shape}, {y_train.shape}")
print(f"Testing set size (X_test, y_test): {X_test.shape}, {y_test.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 3)

Training set size (X_train, y_train): (3068, 512), (3068, 3)
Testing set size (X_test, y_test): (767, 512), (767, 3)



In [ ]:
from sklearn.model_selection import train_test_split

num_participants_dim_p = len(PDim_response_dfs)
if master_human_responses_df_dim_p.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_dim_p):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_dim_p = np.array(X_list)

# Extract y from the concatenated DataFrame
y_dim_p = master_human_responses_df_dim_p[['positive', 'relaxed', 'awake']].values

print(f"Shape of X (features) after implicit alignment: {X_dim_p.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_dim_p.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_dim_p.shape[0] != y_dim_p.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_dim_p, X_test_dim_p, y_train_dim_p, y_test_dim_p = train_test_split(
    X_dim_p, y_dim_p, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_dim_p.shape}, {y_train_dim_p.shape}")
print(f"Testing set size (X_test, y_test): {X_test_dim_p.shape}, {y_test_dim_p.shape}\n")

Shape of X (features) after implicit alignment: (3953, 512)
Shape of y (labels) after implicit alignment: (3953, 3)

Training set size (X_train, y_train): (3953, 512), (3162, 3)
Testing set size (X_test, y_test): (791, 512), (791, 3)



In [ ]:
from sklearn.model_selection import train_test_split

num_participants_disc_i = len(IDisc_response_dfs)
if master_human_responses_df_disc_i.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_disc_i):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_disc_i = np.array(X_list)

# Extract y from the concatenated DataFrame
y_disc_i = master_human_responses_df_disc_i[["happiness", "sadness", "anger", "tenderness", "fear"]].values

print(f"Shape of X (features) after implicit alignment: {X_disc_i.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_disc_i.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_disc_i.shape[0] != y_disc_i.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_disc_i, X_test_disc_i, y_train_disc_i, y_test_disc_i = train_test_split(
    X_disc_i, y_disc_i, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_disc_i.shape}, {y_train_disc_i.shape}")
print(f"Testing set size (X_test, y_test): {X_test_disc_i.shape}, {y_test_disc_i.shape}\n")

Shape of X (features) after implicit alignment: (3894, 512)
Shape of y (labels) after implicit alignment: (3894, 5)

Training set size (X_train, y_train): (3894, 512), (3115, 5)
Testing set size (X_test, y_test): (779, 512), (779, 5)



In [ ]:
from sklearn.model_selection import train_test_split

num_participants_disc_p = len(PDisc_response_dfs)
if master_human_responses_df_disc_p.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_disc_p):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_disc_p = np.array(X_list)

# Extract y from the concatenated DataFrame
y_disc_p = master_human_responses_df_disc_p[["happiness", "sadness", "anger", "tenderness", "fear"]].values

print(f"Shape of X (features) after implicit alignment: {X_disc_p.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_disc_p.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_disc_p.shape[0] != y_disc_p.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_disc_p, X_test_disc_p, y_train_disc_p, y_test_disc_p = train_test_split(
    X_disc_p, y_disc_p, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_disc_p.shape}, {y_train_disc_p.shape}")
print(f"Testing set size (X_test, y_test): {X_test_disc_p.shape}, {y_test_disc_p.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 5)

Training set size (X_train, y_train): (3835, 512), (3068, 5)
Testing set size (X_test, y_test): (767, 512), (767, 5)



# Train regression head (=MLP, a few projection layers)

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor.fit(X_train, y_train)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 13.62211751
Validation score: -4.845158
Iteration 2, loss = 10.04445949
Validation score: -2.650845
Iteration 3, loss = 5.38956519
Validation score: -0.736191
Iteration 4, loss = 2.89141156
Validation score: -0.239232
Iteration 5, loss = 2.13205520
Validation score: -0.005131
Iteration 6, loss = 1.97791601
Validation score: -0.000666
Iteration 7, loss = 1.97541073
Validation score: 0.000774
Iteration 8, loss = 1.96558808
Validation score: 0.001174
Iteration 9, loss = 1.96385654
Validation score: 0.001448
Iteration 10, loss = 1.96186052
Validation score: 0.002587
Iteration 11, loss = 1.96002453
Validation score: 0.003731
Iteration 12, loss = 1.95869658
Validation score: 0.002972
Iteration 13, loss = 1.95712717
Validation score: 0.004827
Iteration 14, loss = 1.95670580
Validation score: 0.005698
Iteration 15, loss = 1.95534090
Validation score: 0.006526
Iteration 16, loss = 1.95371249
Validation score: 0.005794
Iteration 17, loss = 1

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_dim_p = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_dim_p.fit(X_train_dim_p, y_train_dim_p)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 13.23413847
Validation score: -3.854427
Iteration 2, loss = 9.41035042
Validation score: -1.757577
Iteration 3, loss = 4.78188341
Validation score: -0.408806
Iteration 4, loss = 2.83544391
Validation score: -0.107658
Iteration 5, loss = 2.28459183
Validation score: -0.001256
Iteration 6, loss = 2.24609194
Validation score: -0.001298
Iteration 7, loss = 2.23694916
Validation score: 0.001472
Iteration 8, loss = 2.23067523
Validation score: 0.004054
Iteration 9, loss = 2.22631777
Validation score: 0.003952
Iteration 10, loss = 2.22421016
Validation score: 0.007107
Iteration 11, loss = 2.21954454
Validation score: 0.008751
Iteration 12, loss = 2.21542705
Validation score: 0.011384
Iteration 13, loss = 2.21098812
Validation score: 0.012208
Iteration 14, loss = 2.20769308
Validation score: 0.016174
Iteration 15, loss = 2.20078552
Validation score: 0.017900
Iteration 16, loss = 2.19545732
Validation score: 0.023497
Iteration 17, loss = 2.

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_disc_i = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_disc_i.fit(X_train_disc_i, y_train_disc_i)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 4.75032632
Validation score: -0.648464
Iteration 2, loss = 3.28573504
Validation score: -0.188237
Iteration 3, loss = 2.58855773
Validation score: -0.019636
Iteration 4, loss = 2.37238350
Validation score: -0.003112
Iteration 5, loss = 2.35158676
Validation score: -0.007942
Iteration 6, loss = 2.34775818
Validation score: -0.003438
Iteration 7, loss = 2.34125789
Validation score: -0.002729
Iteration 8, loss = 2.34752027
Validation score: -0.002210
Iteration 9, loss = 2.34334440
Validation score: -0.004617
Iteration 10, loss = 2.34521964
Validation score: -0.007681
Iteration 11, loss = 2.34801067
Validation score: -0.007682
Iteration 12, loss = 2.34799723
Validation score: -0.003591
Iteration 13, loss = 2.34278405
Validation score: -0.002104
Iteration 14, loss = 2.34506658
Validation score: -0.004373
Iteration 15, loss = 2.34297951
Validation score: -0.003851
Iteration 16, loss = 2.34310183
Validation score: -0.002215
Iteration 17, 

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_disc_p = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_disc_p.fit(X_train_disc_p, y_train_disc_p)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 6.85922895
Validation score: -1.073800
Iteration 2, loss = 5.00524394
Validation score: -0.393040
Iteration 3, loss = 3.54003179
Validation score: -0.076653
Iteration 4, loss = 2.95180795
Validation score: -0.002227
Iteration 5, loss = 2.86381733
Validation score: -0.009573
Iteration 6, loss = 2.86145611
Validation score: -0.008328
Iteration 7, loss = 2.85524021
Validation score: -0.004350
Iteration 8, loss = 2.85404222
Validation score: -0.003980
Iteration 9, loss = 2.85214798
Validation score: -0.002456
Iteration 10, loss = 2.85172621
Validation score: -0.004164
Iteration 11, loss = 2.85039049
Validation score: -0.001895
Iteration 12, loss = 2.84829943
Validation score: -0.002216
Iteration 13, loss = 2.84706465
Validation score: -0.002462
Iteration 14, loss = 2.84552874
Validation score: -0.001198
Iteration 15, loss = 2.84369251
Validation score: -0.001050
Iteration 16, loss = 2.84396479
Validation score: -0.001682
Iteration 17, 

# Evaluate

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred = mlp_regressor.predict(X_test)

print(f"\nShape of predictions (y_pred): {y_pred.shape}")
print(f"First 5 actual values (y_test):\n{y_test[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error = np.abs((y_test - y_pred) / y_test) * 100
mape = np.mean(absolute_percentage_error)
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Positive', 'Relaxed', 'Awake']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test[:, i], y_pred[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test.shape[1]):
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test[:, i], y_pred[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence = r2_score(y_test[:, 0], y_pred[:, 0])
print("  valence =", r2_valence)

r2_tension = r2_score(y_test[:, 1], y_pred[:, 1])
print("  tension =", r2_tension)

r2_energy = r2_score(y_test[:, 2], y_pred[:, 2])
print("  energy =", r2_energy)



Shape of predictions (y_pred): (767, 3)
First 5 actual values (y_test):
[[3.96 8.32 7.05]
 [7.11 7.01 8.11]
 [4.68 5.04 5.88]
 [2.08 4.04 7.6 ]
 [6.34 6.43 5.32]]
First 5 predicted values (y_pred):
[[5.230155  5.538272  5.1015053]
 [4.632814  4.9675236 4.716338 ]
 [3.5722604 3.1526282 6.2812924]
 [2.542699  1.7638491 6.9967012]
 [4.813151  4.5105314 6.0440493]]

Mean Absolute Percentage Error (MAPE): 41.51%

Mean Absolute Error (MAE): 1.4120
Root Mean Squared Error (RMSE): 1.7640

Pearson Correlation Coefficients (per dimension):
  Positive Dimension: 0.5183
  Relaxed Dimension: 0.5766
  Awake Dimension: 0.2685
  Average Pearson Correlation across dimensions: 0.4545

R-squared scores:
  valence = 0.2640086960469131
  tension = 0.32698769464510224
  energy = 0.07111311203597359


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_dim_p = mlp_regressor_dim_p.predict(X_test_dim_p)

print(f"\nShape of predictions (y_pred): {y_pred_dim_p.shape}")
print(f"First 5 actual values (y_test):\n{y_test_dim_p[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_dim_p[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error = np.abs((y_test_dim_p - y_pred_dim_p) / y_test_dim_p) * 100
mape_dim_p = np.mean(absolute_percentage_error)
print(f"Mean Absolute Percentage Error (MAPE): {mape_dim_p:.2f}%\n")

# Mean Absolute Error (MAE)
mae_dim_p = mean_absolute_error(y_test_dim_p, y_pred_dim_p)
print(f"Mean Absolute Error (MAE): {mae_dim_p:.4f}")

# Root Mean Squared Error (RMSE)
rmse_dim_p = np.sqrt(mean_squared_error(y_test_dim_p, y_pred_dim_p))
print(f"Root Mean Squared Error (RMSE): {rmse_dim_p:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Positive', 'Relaxed', 'Awake']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_dim_p[:, i]) > 1e-6 and np.std(y_pred_dim_p[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_dim_p[:, i], y_pred_dim_p[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_dim_p.shape[1]):
    if np.std(y_test_dim_p[:, i]) > 1e-6 and np.std(y_pred_dim_p[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_dim_p[:, i], y_pred_dim_p[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence_dim_p = r2_score(y_test_dim_p[:, 0], y_pred_dim_p[:, 0])
print("  valence =", r2_valence_dim_p)

r2_tension_dim_p = r2_score(y_test_dim_p[:, 1], y_pred_dim_p[:, 1])
print("  tension =", r2_tension_dim_p)

r2_energy_dim_p = r2_score(y_test_dim_p[:, 2], y_pred_dim_p[:, 2])
print("  energy =", r2_energy_dim_p)



Shape of predictions (y_pred): (791, 3)
First 5 actual values (y_test):
[[3.16 4.31 2.02]
 [8.86 8.86 8.78]
 [8.14 5.35 3.35]
 [5.02 5.01 5.99]
 [5.04 4.31 6.31]]
First 5 predicted values (y_pred):
[[4.7684608 3.896703  6.3585944]
 [6.712663  6.021616  6.1311827]
 [4.3671265 3.5097275 6.576207 ]
 [4.9616423 4.757864  5.62985  ]
 [3.2127542 3.705935  3.8392522]]

Mean Absolute Percentage Error (MAPE): 48.33%

Mean Absolute Error (MAE): 1.5277
Root Mean Squared Error (RMSE): 1.9071

Pearson Correlation Coefficients (per dimension):
  Positive Dimension: 0.5023
  Relaxed Dimension: 0.4823
  Awake Dimension: 0.4181
  Average Pearson Correlation across dimensions: 0.4676

R-squared scores:
  valence = 0.2519560418319541
  tension = 0.23043575425760077
  energy = 0.17339124873535172


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_disc_i = mlp_regressor_disc_i.predict(X_test_disc_i)

print(f"\nShape of predictions (y_pred): {y_pred_disc_i.shape}")
print(f"First 5 actual values (y_test):\n{y_test_disc_i[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_disc_i[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error_disc_i = np.abs((y_test_disc_i - y_pred_disc_i) / y_test_disc_i) * 100
mape_disc_i = np.mean(absolute_percentage_error_disc_i)
print(f"Mean Absolute Percentage Error (MAPE): {mape_disc_i:.2f}%\n")

# Mean Absolute Error (MAE)
mae_disc_i = mean_absolute_error(y_test_disc_i, y_pred_disc_i)
print(f"Mean Absolute Error (MAE): {mae_disc_i:.4f}")

# Root Mean Squared Error (RMSE)
rmse_disc_i = np.sqrt(mean_squared_error(y_test_disc_i, y_pred_disc_i))
print(f"Root Mean Squared Error (RMSE): {rmse_disc_i:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Happiness', 'Sadness', 'Anger', 'Tenderness', 'Fear']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_disc_i[:, i]) > 1e-6 and np.std(y_pred_disc_i[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_disc_i[:, i], y_pred_disc_i[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_disc_i.shape[1]):
    if np.std(y_test_disc_i[:, i]) > 1e-6 and np.std(y_pred_disc_i[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_disc_i[:, i], y_pred_disc_i[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

from sklearn.metrics import r2_score

# R-squared IDisc
print("\nR-squared scores (IDisc):")
r2_happiness_disc_i = r2_score(y_test_disc_i[:, 0], y_pred_disc_i[:, 0])
print("  happiness =", r2_happiness_disc_i)
r2_sadness_disc_i = r2_score(y_test_disc_i[:, 1], y_pred_disc_i[:, 1])
print("  sadness =", r2_sadness_disc_i)
r2_anger_disc_i = r2_score(y_test_disc_i[:, 2], y_pred_disc_i[:, 2])
print("  anger =", r2_anger_disc_i)
r2_tenderness_disc_i = r2_score(y_test_disc_i[:, 3], y_pred_disc_i[:, 3])
print("  tenderness =", r2_tenderness_disc_i)
r2_fear_disc_i = r2_score(y_test_disc_i[:, 4], y_pred_disc_i[:, 4])
print("  fear =", r2_fear_disc_i)



Shape of predictions (y_pred): (779, 5)
First 5 actual values (y_test):
[[1.   1.   1.   1.   1.  ]
 [4.23 6.82 5.75 5.13 5.69]
 [1.28 1.24 1.15 1.18 1.23]
 [1.   1.   2.4  1.   1.54]
 [1.36 4.73 1.07 3.37 1.94]]
First 5 predicted values (y_pred):
[[2.183311  3.1578834 2.7886243 2.5878997 2.9253645]
 [2.021361  2.8021302 4.4539747 2.4340699 3.3574173]
 [3.0520277 2.351227  2.1044042 3.044608  2.4955077]
 [1.5941755 3.2139447 5.1077094 2.181713  3.8207238]
 [2.5480626 3.0476525 2.478311  2.8582978 2.4167411]]

Mean Absolute Percentage Error (MAPE): 87.27%

Mean Absolute Error (MAE): 1.6514
Root Mean Squared Error (RMSE): 2.0733

Pearson Correlation Coefficients (per dimension):
  Happiness Dimension: 0.3457
  Sadness Dimension: 0.2059
  Anger Dimension: 0.4412
  Tenderness Dimension: 0.2135
  Fear Dimension: 0.2370
  Average Pearson Correlation across dimensions: 0.2887

R-squared scores (IDisc):
  happiness = 0.11842500230088782
  sadness = 0.036779122525582464
  anger = 0.18895503543

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_disc_p = mlp_regressor_disc_p.predict(X_test_disc_p)

print(f"\nShape of predictions (y_pred): {y_pred_disc_p.shape}")
print(f"First 5 actual values (y_test):\n{y_test_disc_p[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_disc_p[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error_disc_p = np.abs((y_test_disc_p - y_pred_disc_p) / y_test_disc_p) * 100
mape_disc_p = np.mean(absolute_percentage_error_disc_p)
print(f"Mean Absolute Percentage Error (MAPE): {mape_disc_p:.2f}%\n")

# Mean Absolute Error (MAE)
mae_disc_p = mean_absolute_error(y_test_disc_p, y_pred_disc_p)
print(f"Mean Absolute Error (MAE): {mae_disc_p:.4f}")

# Root Mean Squared Error (RMSE)
rmse_disc_p = np.sqrt(mean_squared_error(y_test_disc_p, y_pred_disc_p))
print(f"Root Mean Squared Error (RMSE): {rmse_disc_p:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Happiness', 'Sadness', 'Anger', 'Tenderness', 'Fear']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_disc_p[:, i]) > 1e-6 and np.std(y_pred_disc_p[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_disc_p[:, i], y_pred_disc_p[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_disc_p.shape[1]):
    if np.std(y_test_disc_p[:, i]) > 1e-6 and np.std(y_pred_disc_p[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_disc_p[:, i], y_pred_disc_p[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

from sklearn.metrics import r2_score

# R-squared PDisc
print("\nR-squared scores (PDisc):")
r2_happiness_disc_p = r2_score(y_test_disc_p[:, 0], y_pred_disc_p[:, 0])
print("  happiness =", r2_happiness_disc_p)
r2_sadness_disc_p = r2_score(y_test_disc_p[:, 1], y_pred_disc_p[:, 1])
print("  sadness =", r2_sadness_disc_p)
r2_anger_disc_p = r2_score(y_test_disc_p[:, 2], y_pred_disc_p[:, 2])
print("  anger =", r2_anger_disc_p)
r2_tenderness_disc_p = r2_score(y_test_disc_p[:, 3], y_pred_disc_p[:, 3])
print("  tenderness =", r2_tenderness_disc_p)
r2_fear_disc_p = r2_score(y_test_disc_p[:, 4], y_pred_disc_p[:, 4])
print("  fear =", r2_fear_disc_p)



Shape of predictions (y_pred): (767, 5)
First 5 actual values (y_test):
[[2.   6.04 1.86 1.01 1.01]
 [1.   6.06 1.   2.99 1.  ]
 [1.   2.01 4.02 1.   1.  ]
 [1.81 5.81 4.99 3.35 1.61]
 [1.26 3.29 2.45 1.26 6.45]]
First 5 predicted values (y_pred):
[[1.9559134 5.301934  3.2101321 2.9371305 3.2248926]
 [1.6498171 4.7751856 4.1861525 2.6898484 4.2375026]
 [2.2055335 4.353793  3.8316903 2.6633449 3.9368265]
 [1.7110813 4.165774  5.3385315 2.0155637 5.631412 ]
 [2.780662  4.0627236 2.905801  3.0510921 3.135839 ]]

Mean Absolute Percentage Error (MAPE): 89.86%

Mean Absolute Error (MAE): 1.8287
Root Mean Squared Error (RMSE): 2.2189

Pearson Correlation Coefficients (per dimension):
  Happiness Dimension: 0.3874
  Sadness Dimension: 0.3665
  Anger Dimension: 0.3701
  Tenderness Dimension: 0.2827
  Fear Dimension: 0.3844
  Average Pearson Correlation across dimensions: 0.3582

R-squared scores (PDisc):
  happiness = 0.14436606714971
  sadness = 0.13420993797876435
  anger = 0.124962656399337